# Theoretical Computer Sciences Project

#### Sergio Peignier and Théotime Grohens

In [ ]:
import random
import pqdict
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import math 

## 1 - Graph Therory

In this project we will apply graph algorithms to study the gene regulatory network (GRN)
of *Saccharomyces cerevisiae*. This species of yeast, it is a small single-cell eukaryote, with a short generation time, and two possible forms: an haploid one and a diploid one. Moreover, this organism can be easily cultured, and it has an important economic impact since it is extensively used for instance,
in winemaking, baking, and brewing. Due to these characteristics, Saccharomyces cerevisiae
is studied as an important model organism.

In this work we will study the **gene regulatory network** of *Saccharomyces cerevisiae*, using graph theory algorithms. The files that are provided for this project have been used in [MCK+12] , as gold-standards to assess gene regulatory network inference algorithms, and they are the result of biological experiments based on ChIP binding data [MWG + 06], and systematic transcription factor deletions [HKI07]. Hereafter we describe each dataset in details:

- GRN_edges_S_cerevisiae.txt : contains the edges of the *S. cervisiae* regulatory network (from transcription factors to target genes). The intended meaning is that if there is an edge between transcription factor X and the target gene A, then X regulates the transcription of A ;

- net4_transcription_factors.tsv : Is a file containing in a single column the identifiers of the transcription factors of *S. cervisiae* that were studied ;

- net4_gene_ids.tsv : The two previous files, use specific identifiers to denote genes, and this file contains the gene name associated to each gene identifier ;

- go_slim_mapping.tab.txt : Only columns 0 and 5 will be used in this work. Column 0 contains the gene name, and column 5 contains its Gene Ontology (GO) annotation (http://www.geneontology.org/). Notice that two different rows may give for the same gene different Gene Ontology annotations. 

### Exercise 1 :  Exploration and characterization of the gene regulatory network

#### 1) Load the dataset and create a NetworkX graph instance.

On importe les datasets avec pandas : 

In [ ]:
GRN_edges_SC = pd.read_csv("./datas/GRN_edges_S_cerevisiae.txt", sep = ',',  header=0)
net4_transcription_factors = pd.read_csv("./datas/net4_transcription_factors.tsv", sep = '\n',  header=0) 
net4_gene_ids = pd.read_csv("./datas/net4_gene_ids.tsv", sep = '\t', header=0) 
go_slim_mappingtab = pd.read_csv("./datas/go_slim_mapping.tab.txt", sep = '\t', header=None) 

On vérifie que tout a bien été importé 

In [ ]:
GRN_edges_SC = GRN_edges_SC.iloc[:,1:]

In [ ]:
# on peut transformer le df en array (au cas où si besoin)
GRN_edges_SC_np = GRN_edges_SC.to_numpy()

In [ ]:
net4_gene_ids.head()

In [ ]:
net4_transcription_factors.head()

In [ ]:
go_slim_mappingtab.head()

In [ ]:
GRN_edges_SC.head()

Le dataset a bien été importé, on créé donc un graphe $G = <V,E>$ dont l'ensemble des noeuds noté $V$ contient les facteurs de transcription et les gènes cibles et l'ensemble des arettes noté $E$ représente les régulations des gènes par les facteurs de transcription.

In [ ]:
G = nx.from_pandas_edgelist(GRN_edges_SC, "transcription_factor", "target_gene")

#### 2) Plot the gene regulatory network,  the plot should be readable,  understandable,  andinformative.  Which information did you decide to convey in your plot?  Why?

On représente le graphe G tel quel.

In [ ]:
#Premiere impression pour le graphe 
plt.figure(figsize=(15,8))
nx.draw(G, node_size = 12)

In [ ]:
#On transforme G en graphe dirigé (pas super utile et prend bcp de temps à charger)
#G = nx.DiGraph(G)

In [ ]:
#On rend le graphe plus lisible 
plt.figure(figsize=(18,10))
pos = nx.spring_layout(G, k = 0.6,) #return the relative positions of the nodes,k = optimal distance between nodes
nx.draw(G, node_size = 18, 
        pos = pos, 
        width = 0.4, 
        node_color = 'cyan',
        edge_color = 'DarkSlateGray')

Le graphe n'est pas lisible tel quel et ne donne aucune information sur la nature des données.

L'ensemble $V$ peut être séparé en deux sous-ensembles :

- $X$ : ensemble des facteurs de transcription ;
- $A$ : ensemble des gènes.

Il serait donc plus 
isdjocospjdkpl de représenter les deux sous-ensembles $X$ et $A$ de façon distincte. 

##### Graphe bipartite :

In [ ]:
from networkx.algorithms import bipartite

In [ ]:
g = nx.Graph()

In [ ]:
g.add_nodes_from(GRN_edges_SC['transcription_factor'], bipartite = 'transcription_factor')

In [ ]:
g.add_nodes_from(GRN_edges_SC['target_gene'], bipartite = 'target_gene')

In [ ]:
g.add_edges_from(zip(GRN_edges_SC['transcription_factor'], GRN_edges_SC['target_gene']))

In [ ]:
#tf = transcription_factor
tf_nodes = [ n for n in g.nodes() if g.nodes[n]
            ['bipartite'] == 'transcription_factor']

In [ ]:
# gn = gene = target_gene
gn_nodes = [ n for n in g.nodes() if g.nodes[n]
           ['bipartite'] == 'target_gene']

In [ ]:
pos_ = nx.bipartite_layout(g, tf_nodes, scale = 1)

Sur le graphe si-dessous, on réprésente la bipartie du graphe avec :

- À gauche les noeuds représentant les facteurs de transcription;
- À droite les noeuds représentant les gène cible pour une régulation.

Globalement (sur une vue d'ensemble) on remarque tout de suite que certain facteurs agissent sur un plus grand nombre de gène cibles que d'autres.

In [ ]:
plt.figure(figsize=(30,20))
nx.draw(g, pos = pos_, node_size = 14,
       node_color = 'forestgreen',
       edge_color = 'darkblue',
       width = 0.1)

#### 3) Describe the network by computing pertinent local and global metrics,  explain your choices, represent the results graphically if necessary, and interpret the results.

Clustering coeff : en fait ça sert à R je crois vu kon a un bipartire

In [ ]:
tf_clust = nx.clustering(G, tf_nodes)
gn_clust = nx.clustering(G, gn_nodes)
clustering = [[k for k in tf_clust.values()], [k for k in gn_clust.values()]]

In [ ]:
sns.set_style("whitegrid")

plt.figure(figsize=(18,10))
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Clustering Coefficient",fontsize=20)
plt.ylabel("Nodes",fontsize=20)
plt.title("Histogram of clustering coefficients", fontsize=20)

x1 = clustering[1]
x2 = clustering[0]
plt.hist([x1, x2], color = ['coral', 'mediumorchid'], 
         edgecolor = 'black', label = ["gene's nodes", "TGF's nodes"])

plt.legend()
plt.show()

In [ ]:
sns.set_style("whitegrid")

plt.figure(figsize=(18,10))
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Clustering Coefficient",fontsize=20)
plt.ylabel("Nodes",fontsize=20)
plt.title("Histogram of clustering coefficients", fontsize=20)

x1 = clustering[1]
plt.hist(x1, color = ['coral'], 
         edgecolor = 'black', label = ["gene's nodes"])

plt.legend()
plt.show() 

In [ ]:
 sns.set_style("whitegrid")

plt.figure(figsize=(18,10))
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Clustering Coefficient",fontsize=20)
plt.ylabel("Nodes",fontsize=20)
plt.title("Histogram of clustering coefficients", fontsize=20)

x2 = clustering[0]
plt.hist(x2, color = ['blue'], 
         edgecolor = 'black', label = ["TGF's nodes"])

plt.legend()
plt.show()

In [ ]:
plt.hist(clustering[1], alpha=0.6)
plt.hist(clustering[0], alpha=0.6)
plt.show()

In [ ]:
print(np.mean(clustering[0]))
print(np.mean(clustering[1]))

In [ ]:
nx.average_clustering(G)

In [ ]:
def overlap (G):
    out = []
    for couple in nx.utils.pairwise(G.nodes()):
        n = 0
        for nei in G[couple[0]]:
            if nei in G[couple[1]]:
                n+=1
        if n == 0:
            O = 0
        elif  G.degree(couple[0]) <= 1 or G.degree(couple[1]) <= 1:
            O = 0
        elif (G.degree(couple[0])-1+G.degree(couple[1])-1-n) == 0:
            print(G.degree(couple[0]),G.degree(couple[1]),n)
            O=100
        else :
            O =  n/(G.degree(couple[0])-1+G.degree(couple[1])-1-n)
        out.append((couple, O))
    return out

In [ ]:
overlap(G)

In [ ]:
nx.closeness_centrality(G) # inverse de la dist moyenne entre n et les autre noeuds

In [ ]:
nx.betweenness_centrality(G) # fraction of shortest paths that passes through n

In [ ]:
nx.density(G)

In [ ]:
RC = nx.rich_club_coefficient(G)

In [ ]:
lists = sorted(RC.items()) # sorted by key, return a list of tuples

x, y = zip(*lists) # unpack a list of pairs into two tuples

plt.plot(x, y)
plt.xlabel("Degree k",fontsize=10)
plt.ylabel("Rich Club Coefficient",fontsize=10)
plt.show()

In [ ]:
def plot_degree_dist(G):
    degrees = [G.degree(n) for n in G.nodes()]
    plt.hist(degrees)
    plt.show()

In [ ]:
degree_freq = nx.degree_histogram(G)
degrees = range(len(degree_freq))
plt.figure(figsize=(12, 8)) 
plt.loglog(degrees, degree_freq) 
plt.xlabel('Degree')
plt.ylabel('Frequency')

#### 4) Implement and apply the k-shell decomposition algorithm.

We can first try to make a k-shell with a function `k_shell` directly implemented in the `networkx.algorithms.core` library, for different $k$.

In [ ]:
from networkx.algorithms.core import k_shell

In [ ]:
nx.draw(nx.k_shell(G, k=2), node_size = 10)

In [ ]:
nx.draw(k_shell(G, k=4), node_size = 10)

In [ ]:
nx.draw(k_shell(G, k=6), node_size = 10)

In [ ]:
nx.draw(k_shell(G, k=7), node_size = 10)

For $k > 7$ there is no more node left.

Yet we try to implement ourself the `k_shell` function.

In [ ]:
def my_kshell (G, k) :
    degrees = []
    GG = nx.Graph()
    GG.add_nodes_from(g)
    GG.add_edges_from(g.edges)
    
    ik = 1
    for ik in range(k):
        done = False
        while not done :
            rm = []
            for n in GG.nodes():
                if GG.degree(n)<=ik:
                    rm.append(n)
            for m in rm:
                GG.remove_node(m)
            done = True
            for n in GG.nodes():
                if GG.degree(n)<=ik:
                    done = False
                    break
            
    return GG

In [ ]:
new_G = my_kshell(G, 7)

In [ ]:
nx.draw(new_G, node_size = 10)

#### 5) For at least 4 of the metrics that you have used:  what is the time complexity of the algorithm that calculates it (explain)?

### Exercise 2 : Community detection

#### 1) You can choose between the Girvan Newman method and the Louvain algorithm tofind communities in the graph. Describe both algorithms, and their time complexities (explain).

#### The Girvan Newman method :

We can express Girvan-Newman algorithm in the following procedure:
- Calculate edge betweenness for every edge in the graph ;
- Remove the edge with highest edge betweenness ;
- Calculate edge betweenness for remaining edges ;
- Repeat steps 2–4 until all edges are removed.

In its simplest and fastest form - worst-case time $O(E^{2}V$) on a network with $E$ edges and $V$ vertices, or $O(E^{3})$ on a sparse graph (where $V = E$).


In [ ]:
#from networkx.algorithms.community.centrality import girvan_newman
#partition_girvan_newman = girvan_newman(G)
#list(partition_girvan_newman)

#### The Louvain algorithm :

The method consists of two phases :
- The first step is to look for "small" communities by optimizing modularity in a local way, where the modularity quantifies the quality of an assignment of nodes to communities. 
- The second step consist of an aggregation of nodes from the same community and to build a new network whose nodes are the communities. 

These steps are repeated iteratively until a maximum of modularity is attained.

###### The algorithm is:

- Start with each node being a singleton cluster: 
- Consider nodes in random order.
- Iterate as long as cluster membership changes 
     - for each node : remove it from its current cluster and add it to the cluster with the highest modularity gain 
- aggregate the resulting clustering to a new graph and continue on next level (step 1), as long as modularity improves.


The time complexity of the Louvain algorithm is $O(V \log(V))$

#### 2) Which algorithm did you choose, why ?

#### 3) For the the Girvan Newman method, the user should select one of the output partitions, explain the criterion that could be used to make this choice, and its complexity.

#### 4) Study the GO composition of each community.  To do this you can produce a countingmatrix $M$,  such  that $M_{i,j}$ is  the  number  of  genes  from  community $j$ that  have  GO annotation $i^1$.

#### 5) Is there a relationship between graph communities and particular cell functions ?

## 2 - Around the Traveling Salesman Problem
### 2.1 - Exact solution

In a complete graph, every node is adjacent to every other node. Therefore, if we take all the nodes in a complete graph in any order, there will be a path through those nodes in that order. (Then, if we join either end of that path, it will give us a Hamilton path.) ?

Let's define an Hamiltonian path $(n_0, n_1, n_2,...,n_n)$ in a complete graph of size $n$. There are $n$ possible starting nodes $n_0$. Then, as one of the $n$ nodes has already been visited, only $n-1$ options are left for $n_1$, $n-2$ for $n_2$ and so on until there is only one node left unvisited : $n_n$. Thus, there are $n\times n-1 \times n-2 \times ... \times 1 = n!$ distinct Hamiltonian paths in a complete graph of size $n$.


In fact, not all these path possibilities are different. On any such cycle, there are:
- $n$ differents nodes where you can start the path ;
- The path is reversible, you can travel it through $2$ directions.

So any one of these $n!$ possible paths is in a set of $2n$ cycles which all contain the same set of edges.
So finaly get $\frac{n!}{2n} = \frac{(n-1)!}{2}$ distinct Hamilton paths



In [ ]:
# Our complet graph (just a test for folowing algorithms)

Gc = nx.Graph()
Gc = nx.complete_graph(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
list(Gc.nodes())

In [ ]:
def add_weight(G) :
    for e in G.edges():
        G[e[0]][e[1]]['weight'] = random.randrange(0,10)+1
        #print(e, Gc[e[0]][e[1]]['weight'])

In [ ]:
def draw_weight(G):
    wwidth = [d['weight'] for (u, v, d) in G.edges(data=True)]

    pos = nx.spring_layout(G)  # positions for all nodes

    # nodes
    nx.draw_networkx_nodes(G, pos)

    # edges
    nx.draw_networkx_edges(G, pos, width=wwidth)

    # labels
    nx.draw_networkx_labels(G, pos, font_size=20, font_family="sans-serif")

    plt.axis("off")
    plt.show()

In [ ]:
add_weight(Gc)

In [ ]:
draw_weight(Gc)

In [ ]:
Gtest = nx.petersen_graph()

In [ ]:
add_weight(Gtest)

In [ ]:
draw_weight(Gtest)

In [ ]:
def succ(G, si, pred) : # returns successors of si that are not in pred
 
    si_succ_init = [s for s in G[si]]
    si_succ = []
    
    for sj in si_succ_init :
        if sj not in pred :
            si_succ.append(sj)
            
    return si_succ

In [ ]:
def next_step(G, si, pred, path, weight, path_weight) :
    if len(pred)>0 :
        sp = pred[len(pred)-1]
        si_w = G[sp][si]['weight']
        weight = weight + si_w
    pred = pred + [si] # si is added to the predecessor list
    si_succ = succ(G, si, pred) # successors of si that are not in pred
    
    for sj in si_succ :
        next_step(G, sj, pred, path, weight, path_weight)
    
    # when each branch has been visited as deep as possible, the list of predecessor is added to path
    if len(si_succ) == 0 :
        path.append(pred)
        path_weight.append((pred, weight))
    
    return None

In [ ]:
def find_path(G, s0) :
    pred = []   
    path = []
    path_weight = []
    weight = 0
    
    next_step(G, s0, pred, path, weight, path_weight)
        
    return path_weight

In [ ]:
def ham_path_filt(G, path_w) :
    # filters out all paths that passes through less nodes than n, with n the number of nodes in G
    # G : networkX graph
    # path_w : list of tuples containing the paths to filter and there weight
    
    n = len(G.nodes())
    ham_path = []
    
    for tup in path_w :
        if len(tup[0]) >= n :
            ham_path.append(tup)
            
    return ham_path

In [ ]:
def Hamiltonian_path(G, s0) :
          
    return ham_path_filt(G, find_path(G, s0))

In [ ]:
Hamiltonian_path(Gtest, 1)

In [ ]:
Gc_path = Hamiltonian_path(Gc, 'a')
Gc_path

In [ ]:
N_Gc_path = len(Gc_path)
n = len(Gc.nodes())
print(N_Gc_path == math.factorial(n-1)) 

As we showed before, we can find $n!$ Hamiltonian paths in a complete graph of size $n$. Here we know the starting node $n_1$ so we don't have the $n$ options for $n_1$ but only one, so we end up with $\frac{n!}{n} = (n-1)!$ Hamiltonian paths.

In [ ]:
def shortest_path(path_w) :
    w_min = path_w[0][1]
    path_min = []
    
    for tup in path_w :
        if tup[1] < w_min :
            w_min = tup[1]
            path_min = [tup]
        elif tup[1] == w_min :
            path_min.append(tup)
            
    return path_min

In [ ]:
def shortest_ham_path(G, s0) :
    path_w = Hamiltonian_path(G, s0)
    path_min = shortest_path(path_w)
            
    return path_min
    

In [ ]:
Gc_min_path = shortest_ham_path(Gc, 'a')
Gtest_min_path = shortest_ham_path(Gtest, 1)

In [ ]:
print('Gc_min_path', Gc_min_path)
print('Gtest_min_path', Gtest_min_path)

In the research of all Hamiltonian paths, `si_succ`, on which we loop, gets smaller as we go deeper. For `si = s0`, the maximum size of `si_succ` is $n$ (if we can have an edge that liks `si` to itself), then as we go a step deeper, the maximum size of `si_succ` is $n-1$, then $n-2$, and so on. The time complexity of the method `Hamiltonian_path` is $O(n!)$.

In [ ]:
def Traveling_Salesman(G) :
    path_w = []
    for s in G.nodes() :
        path_w = path_w + Hamiltonian_path(G, s)
    
    path_min = shortest_path(path_w)
    
    return path_min

In [ ]:
Traveling_Salesman(Gc)

In [ ]:
Traveling_Salesman(Gtest)

### 2.2 - The Nearest Neighbor heuristic

In [ ]:
def nearest_neighbour(G, s0) :
    # Implementation of the nearest neighbour heuristic
    # Parameters :
    # G : weighted networkX graph
    # s0 : node in G
    # Return value : tuple
    # list of nodes, shortest Hamiltonian path starting at s0
    # int, total weight of the path
    # list of the nodes left unvisited
    
    shortest_path = [] # returned path
    spath_w = 0 # total weight of the path
    n = G.number_of_nodes() 
    
    si = s0 # active node, initialised as s0
    shortest_path.append(si) # si is added to the path
    
    while len(shortest_path) < n : # while there are still unvisited nodes in G
        # the neighbors of si that has not been visited yet are stored in succ
        succ = [] 
        for sj in G[si] :
            if sj not in shortest_path :
                succ.append(sj)
        
        succ_weight = [(s, G[si][s]['weight']) for s in succ] # weight of the edge linking si to its neighbors
        
        if len(succ) == 0 : # If there are no successor
            return (shortest_path, spath_w, G.nodes()-shortest_path)
        
        # fiding the edge with the minimal weight
        min_w = succ_weight[0][1] # minimal weight initialisation
        min_s = succ_weight[0][0] # corresponding successor node
        
        for tupNW in succ_weight :
            if tupNW[1] < min_w : # if a lower weight is found
                min_w = tupNW[1] # min_w actualisation
                min_s = tupNW[0] # min_s actualisation
       
        shortest_path.append(min_s) # the closest successor is added to the path
        spath_w = spath_w + min_w
        si = min_s # the closest successor is now the active node
    
    return (shortest_path, spath_w)

In [ ]:
nearest_neighbour(Gtest, 5)

In [ ]:
nearest_neighbour(Gc, 'f')

Time complexity of each loop :

- `while len(shortest_path) < n :` $n$
    - `for sj in G[si] :` $n$
    - `for tupNW in succ_weight :` $n$
    
So the time complexity of `nearest_neighbour` is $O(n \times (n+n)) = O(2n^2)$.

In [ ]:
Gex = nx.Graph()
Eex = [(0, 1, {'weight': 4}), (0, 4, {'weight': 4}), (0, 5, {'weight': 2}), (1, 2, {'weight': 8}), (1, 6, {'weight': 1}), (2, 3, {'weight': 5}), (2, 7, {'weight': 4}), (3, 4, {'weight': 8}), (3, 8, {'weight': 8}), (4, 9, {'weight': 10}), (5, 7, {'weight': 6}), (5, 8, {'weight': 8}), (6, 8, {'weight': 5}), (6, 9, {'weight': 2}), (7, 9, {'weight': 7})]
Gex.add_edges_from(Eex)

In [ ]:
nearest_neighbour(Gex, 9)

In [ ]:
Traveling_Salesman(Gex)

In [ ]:
nearest_neighbour(Gex, 8)

But it fails to find the shortest Hamiltonian path in some cases, for example, in `Gex`, with `9` as a starting node, this method does not create an Hamiltonian path. And for `8` as a starting node, it creates an Hamiltonian path but not the shortest.

### 2.3 - The Minimum Spanning Tree heuristic (MST)

#### Using an appropriate data structure, what is the time complexity of this algorithm?

If an **adjacency list** is used to represent the graph, using a  **Breadth First Search (BFS)**, all the vertices can be browsed in  $O(V + E)$ time.

Then we can use a **min heap** as a priority queue to store vertices not yet included in the MST and to get the minimum weight edge.

A Heap is a special Tree-based data structure in which the tree is a complete binary tree. In a Min-Heap the key present at the root node must be minimum among the keys present at all of it’s children. The same property must be recursively true for all sub-trees in that Binary Tree.

A **min heap** opération has a complexity time of $O(\log(V))$

Thus we have a total time compelxity of :
\begin{align*}
    & = O(V + E) \times O(\log(V)) \\
    & = O((V + E)\log(V)) \\
\end{align*}

#### Prove that during each iteration of the while loop, the subgraph (W, F ) is a tree.

##### Initialisation :
For the first iteration, $W = W_0 = \{x_0\}$ and $F = F_0 = \emptyset$. 

> Let $(x, y) \in E$ the shortest edge such that $x \in W$ and $y \notin W$

- $x_0$ is the only element that is in $W_0$ so $x = x_0$ ;

- let $x_1$ the closest neighbor of $x_0$, $y = x_1$.

> $W \gets W \cup y$

> $F \gets F \cup (x, y)$

We now have $W = W_1 = \{x_0, x_1\}$ and $F = F_1 = \{(x_0, x_1)\}$.

$(W, F) = (W_1, F_1)$ consist of only two nodes linked by one edge so it's an acyclic and connected graph, we can say that we have a tree at the end of this first iteration.

##### Heredity :
Let's suppose that at the end of the $n^{th}$ iteration we have a tree $(W, F) = (W_n, F_n)$ with $W_n = \{ x_0, x_1, ..., x_n\}$.

> Let $(x, y) \in E$ the shortest edge such that $x \in W$ and $y \notin W$

- let $(x_i, x_{n+1}) \in E$ the shortest edge such that $x_i \in W_n$ and $x_{n+1} \notin W_n$ ;
- we thus have $x = x_i$ and $y = x_{n+1}$.

> $W \gets W \cup y$

> $F \gets F \cup (x, y)$

At the end of this $(n+1)^{th}$ iteration we have $(W, F) = (W_{n+1}, F_{n+1})$ with $W_{n+1} = \{ x_0, x_1, ..., x_n, x_{n+1}\}$ and $F_{n+1} = F_n \cup \{(x_i, x_{n+1})\}$.

As $(W_n, F_n)$ is a tree and that we created $(W_{n+1}, F_{n+1})$ by adding one node $x_{n+1}$ and connecting it to the tree $(W_n, F_n)$ with one edge $(x_i, x_{n+1})$, $(W_{n+1}, F_{n+1})$ is connected and is acyclic because we didn't add any edge between two nodes of $(W_n, F_n)$.
(
##### Conclusion :
We have shown that :
- at the end of the first iteration of the `while` loop $(W, F)$ ;
- if we have a subgraph $(W, F) = (W_n, F_n)$ that is a tree as a precondition to any iteration, we end up with the subgraph $(W, F) = (W_{n+1}, F_{n+1})$ that is also a tree at the end of the iteration.

We can thus say that, according the recurrence principle, the subgraph $(W, F)$ that we have when the exiting condition of the `while` loop $W \neq V$ is met is a tree and that $(W, F) = (V, F)$. So the subgraph $(V, F)$ returned at the end of Prim's algorithm is a tree.

#### Deduce that Prim’s algorithm returns a spanning tree.

#### Write a Python function that implements Prim’s algorithm.

In [ ]:
#Function recives a graph and a starting node, and returns a MST : 

def prims_algo(G,start):
    N = len(G) - 1
    current = start
    visited = set() # We store all the visited nodes
    pq = pqdict.PQDict() #from the module pqdict (priority queue dictionnary) :
    MST = []

    while len(MST) < N:
        for node in G.neighbors(current):
            if node not in visited and current not in visited:
                if (current,node) not in pq and (node,current) not in pq:
                    w = G[current][node]['weight']
                    pq.additem((current,node), w)

        visited.add(current)
        edge_, weight_ = pq.popitem() # We get the last tuple of the priority queue with its weight
        
        while(edge_[1] in visited):
            edge_, weight_ = pq.popitem()
        MST.append(edge_)
        current = edge_[1]

    return MST

In [ ]:
MSTtest = prims_algo(Gtest, 0)
MSTtest

In [ ]:
other_edges = [n for n in Gtest.edges() if n not in MSTtest]
other_edges

In [ ]:
pos = nx.spring_layout(Gtest)

# nodes
nx.draw_networkx_nodes(Gtest, pos, node_size=500)

# edges
nx.draw_networkx_edges(Gtest, pos, edgelist=MSTtest, width=4)
nx.draw_networkx_edges(Gtest, pos, edgelist=other_edges, width=4, alpha=0.5, edge_color='b', style='dashed')

# labels
nx.draw_networkx_labels(Gtest,pos,font_size=20,font_family='sans-serif')

plt.axis('off')
plt.show()

The triangular inequality is the following inequality:
\begin{equation*}
    \forall x, y, z \in V, w(x, z) \leq w(x, y) + w(y, z),
\end{equation*}

where $w(x, y)$ is the weight of edge $x → y$ (more direct paths are shorter).

#### Assuming that the graph verifies the triangle inequality, show that the length of the Hamiltonian cycle obtained by visiting the MST is less than twice the length of the shortest Hamiltonian cycle of the graph.

Take the shortest Hamiltonian cycle called $H_s$ and remove an arbitrary edge. The résult is a spannig tree called $T$. So we can first write the following equality if we consider that all edge have a non-negative weight :


> $length(T) \leq length (H_s)$


Now, let $P$ be the full path of the Minimum Spanning Tree (MST) denoted by $T_{MST}$. This full path will include repeated vertices. 

Example : {0, 1, 2, 1, 7, 1, 0, 3, 4, 5, 6, 4, 3, 0}

So as you can imagine the full path $P$ traverses every edges of the $T_{MST}$ twice. So we have :


> $length(P) = 2 \times length (T_{min})$

> $length(P) = 2 \times length (T_{min}) \leq length(T)$

> $length(P) = 2 \times length (T_{min}) \leq length(T) \leq length(H_s)$

The problem is $P$ is not a proper cycle since come vertices may be visited more than once. So by using the **triangle inequality**, we delete a visit to every vertices from $P$ to obtain a proper Hamiltonian cycle denoted $H$ where the **weight does not increase**.

In our example : {0, 1, 2, 7, 3, 4, 5, 6, 0}

Then we get :


> $length(H) \leq length (P)$

> $length(H) \leq length (P) \leq 2 \times length (T_{min})$

> $length(H) \leq length (P) \leq 2 \times length (T_{min}) \leq length(T) \leq length(H_s)$


So finaly we have :


> $length(H) \leq 2 \times length (H_s)$



QED.